In [1]:
library(dplyr)
library(readr)
library(tidyr)
library(stringr)
library(ggplot2)
library(lubridate)
library(survival)
library(survminer)

Warning message:
“package ‘dplyr’ was built under R version 3.6.3”
Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Warning message:
“package ‘tidyr’ was built under R version 3.6.3”Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang

Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date

Warning message:
“package ‘survminer’ was built under R version 3.6.3”Loading required package: ggpubr
Warning message:
“package ‘ggpubr’ was built under R version 3.6.3”

In [ ]:
##############################
# Creation of Financial Series
##############################
monsanc <- read_csv("monsanc.csv") %>%
  dplyr::select(person_id, case_filed_date, sentence_date,filed_county,
         case_mkey, pubdef, charge_offense, charge_degree, 
         priors, prob_days, conf_days, race, gender,
         name_full, birth_year, birth_date, trial_flag, adj) %>%
  dplyr::mutate(sentence_date = as.character(format(sentence_date, "%Y-%m" )))

In [ ]:
#MNCIS Financial - originally at transaction level
mncis <- read_delim("MNCIS_Financial.txt", delim = "|", na = "NULL",
                    col_types = list(col_double(), col_double(), 
                                     col_double(), col_character(), 
                                     col_character(), col_double(), 
                                     col_datetime(), col_character(),
                                     col_character(), col_double(),
                                     col_double(), col_double(),
                                     col_double(), col_double(),
                                     col_double(), col_double())) %>%
  slice(-dim(.)[1])
str(mncis)

In [ ]:
#Merging reduced category variable
mncis.bridge <- read_csv(file = "mncis_fees.csv")
mncis$Fee_Type_Category_Desc <- str_trim(mncis$Fee_Type_Category_Desc, side = "both") #trimming wtspace
mncis$Fee_Type_Desc <- str_trim(mncis$Fee_Type_Desc, side = "both") #trimming wtspace

mncis <- mncis %>% left_join(mncis.bridge, 
                             by = c("Fee_Type_Desc"="fee_type_desc", "Fee_Type_Category_Desc"="fee_type_category_desc"))

#mncis time series
#mncis financial transaction date for ordered IS the sentence date
mncis.time <- mncis %>%
  select(Case_Mkey, new_cat, 
         Financial_Transaction_Detail_Charge_Amount, 
         Financial_Transaction_Detail_Payment_Amount,
         Financial_Transaction_Detail_Credit_Amount,
         Financial_Transaction_Date) %>%
  rename(mncis_ordered = Financial_Transaction_Detail_Charge_Amount,
         mncis_collected = Financial_Transaction_Detail_Payment_Amount,
         mncis_credit = Financial_Transaction_Detail_Credit_Amount,
         type = new_cat) %>%
  mutate(date = format(as.Date(Financial_Transaction_Date), "%Y-%m")) %>% 
  filter(type!="BAIL" & type!="UNK" & type!="OTHER") %>%
  mutate(mncis_ordered_adj = mncis_ordered-mncis_credit) %>%
  mutate(mncis_ordered_adj = ifelse(mncis_ordered_adj < 0, 0, mncis_ordered_adj),
         mncis_collected = ifelse(mncis_collected < 0, 0, mncis_collected)) %>%
  select(-type, -mncis_ordered) %>%
  group_by(Case_Mkey, date) %>%
  summarise(mncis_ordered = sum(mncis_ordered_adj, na.rm = T),
            mncis_collected = sum(mncis_collected, na.rm=T)) %>%
  arrange(Case_Mkey, date)

rm(mncis.bridge, mncis)


#vibes - made to match mncis.time
#VIBES financial 
vibes <- read_delim("VIBES_Financial.txt", delim = "|", na = "NULL") %>% 
  slice(-dim(.)[1]) 

#vibes time series
#vibes does not have sentence date, must merge in from sentence
vibes.clean <- vibes %>% 
  select(Incident_ID,  
         Total_Assessments, 
         Total_Payments,
         Payment_Date) %>%
  mutate(vibes_ordered = ifelse(Total_Assessments < 0, 0, Total_Assessments),
         vibes_collected = ifelse(Total_Payments < 0, 0, Total_Payments),
         payment_date = format(Payment_Date, "%Y-%m-%d")) %>%
  select(-Payment_Date, -Total_Assessments, -Total_Payments) %>%
  mutate(vibes_ordered = ifelse(is.na(vibes_ordered), 0, vibes_ordered),
         vibes_collected = ifelse(is.na(vibes_collected), 0, vibes_collected))

#merge vibes bridge to vibes, then select unique identifiers merge
vibes.bridge <- read_delim("VIBES_Bridge.txt",  delim = "|", na = "NULL") %>%
  slice(-dim(.)[1]) 

vibes.merge <- vibes %>% left_join(vibes.bridge, by = c("Incident_ID" = "INCIDENT_ID")) %>%
  mutate(county = ifelse(CNTY_CODE==62, "Ramsey County", "Hennepin County")) %>%
  select(Case_Mkey, county, Incident_ID) %>% distinct(Incident_ID, .keep_all = T)

vibes.clean <- vibes.clean %>% left_join(vibes.merge, by = "Incident_ID")

#merge in sentence date from person.case object (have to merge by Case_Mkey and county)
vibes.clean <- vibes.clean %>% 
  left_join(monsanc, by = c("county"="filed_county", "Case_Mkey"="case_mkey")) %>%
  select(Incident_ID, Case_Mkey, vibes_ordered, vibes_collected, sentence_date, payment_date, county) 

#the following sets date to the respective event date, then appends and summarizes over case and date
vibes.ordered <- vibes.clean %>% 
  select(Incident_ID, vibes_ordered, sentence_date) %>%
  rename(date = sentence_date)

vibes.payed <- vibes.clean %>% select(Incident_ID, vibes_collected, payment_date) %>%
  mutate(date = format(as.Date(payment_date), "%Y-%m")) %>% 
  select(-payment_date)

vibes.time <- vibes.ordered %>% bind_rows(vibes.payed) %>%
  group_by(Incident_ID, date) %>%
  summarize(vibes_ordered = sum(vibes_ordered, na.rm=T),
            vibes_collected = sum(vibes_collected, na.rm=T)) %>%
  arrange(Incident_ID, date) %>%
  filter(!is.na(date)) #these are due to missing sentence_dates for cases in vibes series

vibes.time <- vibes.time %>% left_join(vibes.merge, by = "Incident_ID")  

#merging monsanc to mncis and vibes series (have to do merges separately before appending due to different id's)
mncis.person <- mncis.time %>% 
  left_join(monsanc, by = c("Case_Mkey"="case_mkey")) %>% 
  rename(ordered = mncis_ordered, collected = mncis_collected, county = filed_county) 

vibes.person <- vibes.time %>% 
  left_join(monsanc, by = c("county"="filed_county", "Case_Mkey"="case_mkey")) %>%
  rename(ordered = vibes_ordered, collected = vibes_collected) %>% ungroup() %>% select(-Incident_ID) 

#combining - case-level financial series
fin.series <- mncis.person %>% bind_rows(vibes.person) %>% arrange(person_id, date)

#create case count (data already arranged by date)
count <- fin.series %>% select(person_id, Case_Mkey) %>% distinct(person_id,Case_Mkey) %>%
  group_by(person_id) %>% mutate(count = row_number()) 

fin.series <- fin.series %>% left_join(count, by = c("person_id", "Case_Mkey"))

rm(list = ls()[!ls() %in% c("fin.series", "monsanc")])

#adjusting amounts to January 2018, other recodes
fin.series <- fin.series %>%
  mutate(ordered = ordered*adj, 
         collected = collected*adj, 
         prob_days=ifelse(is.na(prob_days), 0, prob_days),
         sentence_year = as.Date(sentence_date, format = "%Y")) %>%
  ungroup()

In [ ]:
############################################
# Fixed Effects Framework
##########################################

#pulling in case level data
monsanc <- read_csv("monsanc.csv") %>%
  select(person_id,case_mkey, case_filed_date, sentence_date, filed_county, adj) %>%
  mutate(sentence_date = as.character(format(sentence_date, "%Y-%m" )))

In [ ]:
#list of cases
### changed 'case_filed_date)%>%s' to 'case_filed_date)%>%'
### in third line because of error in above cell -RS
cases <- monsanc %>%
  select(person_id, case_filed_date, case_mkey) %>%
  rename(event_mkey = case_mkey, event_filed_date = case_filed_date)%>%
  mutate(event_filed_date = substr(event_filed_date, 0, 7))

In [ ]:
#Received and error that 'adj' was not a column name, so I first
#  added 'adj' to select(), but received the same error, so
#  I just removed 'adj' from the entire call - RS
panel <- fin.series %>% 
  filter(case_filed_date >= "2010-01-01") %>%
  select(person_id, date, ordered, collected) %>%
  group_by(person_id, date) %>%
  summarize(ordered = sum(ordered, na.rm = T), 
            collected = sum(collected, na.rm = T)) %>%
  mutate(year = substr(date, 0, 4)) %>%
  left_join(cases, by = c("person_id", "date" = "event_filed_date")) %>%
  mutate(date = as.Date(paste(date, 01, sep="-"), format = "%Y-%m-%d")) %>%
  group_by(person_id) %>%
  complete(date = seq.Date(min(date), as.Date("2018-12-01"), by = "month")) %>%
  ungroup() %>%
  fill(person_id, year) %>%
  mutate(ordered = replace_na(ordered, 0), collected = replace_na(collected, 0)) %>% 
  group_by(person_id) %>%
  mutate(cum_order = cumsum(ordered),
         cum_coll  = cumsum(collected)) %>%
  mutate(lfo_debt = cum_order-cum_coll,
         event = if_else(is.na(event_mkey), 0, 1))

In [8]:
# To save time from having to continually execute the
#   above code while troubleshooting, I saved it as 
#   a csv and just reloaded the csv - RS
panel2 <- read_csv("panel2_df.csv")

Warning message:
“Missing column names filled in: 'X1' [1]”Parsed with column specification:
cols(
  X1 = col_double(),
  event = col_double(),
  lfo_debt = col_double(),
  person_id = col_double()
)


In [9]:
# Limit to only these three colums for now - RS
panel2 <- panel2[,c("event", "lfo_debt", "person_id")]

In [11]:
# Delete problematic person_id - RS
panel2 <- filter(panel2, person_id != 3396157)

In [13]:
fe <- clogit(event~lfo_debt+strata(person_id), 
             method = "efron",
             data = panel2)

In [14]:
summary(fe)

Call:
coxph(formula = Surv(rep(1, 163658970L), event) ~ lfo_debt + 
    strata(person_id), data = panel2, method = "efron")

  n= 163658970, number of events= 2882167 

               coef  exp(coef)   se(coef)      z Pr(>|z|)    
lfo_debt -8.102e-05  9.999e-01  8.902e-07 -91.02   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

         exp(coef) exp(-coef) lower .95 upper .95
lfo_debt    0.9999          1    0.9999    0.9999

Concordance= 0.419  (se = 0 )
Likelihood ratio test= 11635  on 1 df,   p=<2e-16
Wald test            = 8285  on 1 df,   p=<2e-16
Score (logrank) test = 3161  on 1 df,   p=<2e-16
